In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

24/06/26 19:30:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
Players = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Players") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Championships = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Championships") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [12]:
Wimbledon=Championships\
    .select(col("Wimbledon").alias("pid"))
Fr_open=Championships\
    .select("Fr_open")
US_open=Championships\
    .select("US_open")
Au_open=Championships\
    .select("Au_open")

final_df = Wimbledon\
    .unionAll(Fr_open)\
    .unionAll(US_open)\
    .unionAll(Au_open)\
    .groupby("pid")\
    .count()



final_df\
    .join(Players,on=final_df.pid == Players.player_id,how="left")\
    .select("pid","player_name","count")\
    .show()


+---+-----------+-----+
|pid|player_name|count|
+---+-----------+-----+
|  1|      Nadal|    7|
|  2|    Federer|    5|
+---+-----------+-----+

